In [ ]:
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
import sys
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, GRU, Permute, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from google.colab import drive
from model import crnn

In [ ]:
IN_COLAB = 'google.colab' in sys.modules

### Load data

In [ ]:
def load_spectrograms_and_labels(slice_len):

    image_filename = 'specs_{}_sec.npy'.format(slice_len)
    label_filename = 'labels_{}_sec.npy'.format(slice_len)
    
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        data_folder = '/content/drive/ML_with_Eminem/data/'
        print('Loading data from google drive\n')
        
    except:
        data_folder = 'data/'
        print('Loading data from local directory\n')
        
    X = np.load(data_folder+image_filename)
    y = np.load(data_folder+label_filename)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=True)
    
    return X_train, y_train, X_test, y_test, data_folder

### Hyperparameters

In [ ]:
slice_lens = [3]
first_conv_size = [32,64]
other_convs_size = [64, 128, 256]
gru_size = [32, 64]
dense_activation = ['softmax', 'sigmoid']
learning_rate = [0.001, 0.0001]
batch_size = [8, 16, 32]
hyperparams = [first_conv_size, other_convs_size, gru_size, dense_activation, learning_rate, batch_size]
hyper_combos = list(itertools.product(*hyperparams))
len(hyper_combos)

In [ ]:
weights_folder = 'weights/'

runs = pd.DataFrame(columns=['slice_len','first_conv_size', 'other_convs_size', 'gru_size', 'dense_activation',
                             'learning_rate','batch_size', 'train_accuracy', 'val_accuracy', 'train_loss',
                             'val_loss', 'model path'])

for slice_len in slice_lens:
    print('Slice length: {} seconds'.format(slice_len))
    
    # Load data
    X_train, y_train, X_test, y_test, data_folder = load_spectrograms_and_labels(slice_len)
    
    # Save test data for later testing
    np.save(data_folder+'spectrograms_test_{}sec.npy'.format(slice_len),X_test)
    np.save(data_folder+'labels_test_{}_sec.npy'.format(slice_len),y_test)
    
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    print('input shape:', input_shape)
    
    for i in range(len(hyper_combos)):

        # Hyperparameters
        p = hyper_combos[i]
        weights = weights_folder + 'filter1{}_filter2{}_GRU{}_dense_act{}_lr{}_batch{}'.format(p[0],p[1],p[2],p[3],
                                                                                               p[4],p[5])
        print(weights)
        # Create model
        model = crnn(first_conv_size=p[0], other_convs_size=p[1], gru_size=p[2],
                     dense_activation=p[3], input_shape=input_shape)

        model.compile(loss='binary_crossentropy', optimizer=Adam(lr=p[4]), metrics=['accuracy'])
        checkpointer = ModelCheckpoint(filepath=weights, verbose=1, save_best_only=True)
        earlystopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)  

        # Train model
        history = model.fit(X_train, y_train, batch_size=p[5], epochs=1, validation_split=0.1,
                           callbacks=[checkpointer, earlystopper])

        # Save information about hyperparameters and results
        runs.loc[len(runs)] = [slice_len, p[0],p[1],p[2], p[3], p[4], p[5], history.history['accuracy'][-1],
                               history.history['val_accuracy'][-1], history.history['loss'][-1],
                               history.history['val_loss'][-1], weights]
    
runs.to_csv('models/runs.csv')